In [66]:
using LinearAlgebra
include("../test_data.jl")
include("../graph_utils.jl")

load_weight_test_data (generic function with 1 method)

In [68]:
W_Y, D, A_init, coord = load_weight_test_data()
global_efficiency(W_Y)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 87.32124598286491 … 102.9465880930495 114.27160627207444; 87.32124598286491 0.0 … 67.68308503607086 56.52433104425032; … ; 102.9465880930495 67.68308503607086 … 0.0 40.76763422127902; 114.27160627207444 56.52433104425032 … 40.76763422127902 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-56.0 -44.0 5.0; -2.0 21.0 27.0; … ; 45.0 -25.0 11.0; 43.0 12.0 -6.0])

global_efficiency (generic function with 2 methods)

0.3541510943882343

In [ ]:
using Lux
using Random
using Zygote
using Optimisers

In [75]:
import Pkg; Pkg.add("Lux")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed LuxLib ───────────────── v0.3.1
   Installed ADTypes ──────────────── v0.1.6
   Installed Optimisers ───────────── v0.2.19
   Installed Functors ─────────────── v0.4.5
   Installed LuxCore ──────────────── v0.1.4
   Installed LuxDeviceUtils ───────── v0.1.5


   Installed NNlib ────────────────── v0.9.4


   Installed LLVM ─────────────────── v6.1.0


...

  ✓ Distributions → DistributionsChainRulesCoreExt


  ✓ LuxLib


  ✓ GLM


  ✓ KernelDensity


  ✓ LuxLib → LuxLibForwardDiffExt
